In [31]:
# Imports
import os
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
from pathlib import Path

# PyTorch & Lightning
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
import glob

# Scikit-Learn
from sklearn.model_selection import KFold
from sklearn.model_selection import GroupKFold
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder, RobustScaler, PowerTransformer
import timm

In [23]:
class CFG:
    # Data Paths
    TRAIN_PATH = "/kaggle/input/csiro-biomass/train.csv"         # Update this path
    TRAIN_DIR = "/kaggle/input/csiro-biomass/train/"          # Update this path to where images are stored

    # Test paths
    TEST_PATH = "/kaggle/input/csiro-biomass/test.csv"     # Update this path
    TEST_DIR = "/kaggle/input/csiro-biomass/test/"   # Update this path to where test images are stored
    
    # Model Settings
    # 'vit_base_patch14_dinov2.lvd142m' is good, 'vit_small...' is faster
    MODEL_NAME = "vit_base_patch14_dinov2.lvd142m"
    IMG_SIZE = 518                 # DINOv2 uses patches of 14 (224/14 = 16)
    
    # Hyperparameters
    BATCH_SIZE = 16                # Lower if OOM
    EPOCHS = 20
    EPOCHS_FT = 5    
    WEIGHT_DECAY = 1e-2
    N_FOLDS = 5
    SEED = 42
    
    # Differentiated Learning Rates
    LR_BACKBONE = 1e-6                 # Lower LR for DINOv2 Backbone
    LR_HEAD = 1e-4                     # Higher LR for Head and Tabular Branch
    
    # Features & Targets
    TARGET_COLS = ['Dry_Total_g', 'GDM_g', 'Dry_Green_g', 'Dry_Dead_g', 'Dry_Clover_g']

    # Feature weights defined by the competition
    TARGET_WEIGHTS = {
        'Dry_Total_g': 0.5,
        'GDM_g': 0.2,
        'Dry_Green_g': 0.1,
        'Dry_Dead_g': 0.1,
        'Dry_Clover_g': 0.1
    }
    
    NUM_FEATS = ["Pre_GSHH_NDVI", "Height_Ave_cm"]
    CAT_FEATS = ["State", "Species", "season"]

    # Number of last blocks of the DINOv2 backbone to unfreeze during fine-tuning
    NUM_BLOCKS_TO_UNFREEZE = 2

In [24]:
def load_and_preprocess_data(train_path):
    print("📂 Loading and formatting data...")
    train_df_long = pd.read_csv(train_path)
    
    # Extract image ID
    train_df_long['image_id'] = train_df_long['sample_id'].str.split('__').str[0]
    
    # Pivot to Wide Format
    meta_cols = ['image_path', 'Sampling_Date', 'State', 'Species', 
                 'Pre_GSHH_NDVI', 'Height_Ave_cm']
    
    train_df_wide = train_df_long.pivot_table(
        index=['image_id'] + meta_cols,
        columns='target_name',
        values='target',
        aggfunc='first'
    ).reset_index()
   
    # Temporal Features
    train_df_wide['date'] = pd.to_datetime(train_df_wide['Sampling_Date'], format='%Y/%m/%d')
    train_df_wide['month'] = train_df_wide['date'].dt.month
    
    # Season (Southern Hemisphere)
    def get_season(month):
        if month in [9, 10, 11]: return "Spring"
        elif month in [12, 1, 2]: return "Summer"
        elif month in [3, 4, 5]: return "Autumn"
        else: return "Winter"
        
    train_df_wide['season'] = train_df_wide['month'].apply(get_season)
    
    # Construct full image path
    # Adjust extension (.png/.jpg) based on your actual data
    train_df_wide['full_image_path'] = train_df_wide['image_id'].apply(lambda x: os.path.join(CFG.TRAIN_DIR, f"{x}.png"))
    
    print(f"✅ Loaded {len(train_df_wide)} unique train samples.")
    return train_df_wide

In [25]:
class BiomassPreprocessor:
    """
    Handles scaling of inputs and Log-Transform of targets.
    Fits ONLY on train data to prevent leakage.
    """
    def __init__(self):
        self.pt = PowerTransformer(method="yeo-johnson")
        self.scaler = RobustScaler()
        self.encoder = OneHotEncoder(sparse_output=False, handle_unknown="ignore")
        self.cat_cols = CFG.CAT_FEATS
        self.num_cols = CFG.NUM_FEATS
        
    def fit_transform(self, df):
        # 1. Numeric: Power Transform -> Robust Scale
        X_num = self.pt.fit_transform(df[self.num_cols])
        X_num = self.scaler.fit_transform(X_num)
        
        # 2. Categorical: One Hot
        X_cat = self.encoder.fit_transform(df[self.cat_cols])
        
        # Combine
        return np.hstack([X_num, X_cat])

    def transform(self, df):
        # Use fitted scalers
        X_num = self.scaler.transform(self.pt.transform(df[self.num_cols]))
        X_cat = self.encoder.transform(df[self.cat_cols])
        return np.hstack([X_num, X_cat])
    
    def transform_targets(self, df):
        # Log1p transform to handle skew/outliers in biomass
        # y = log(x + 1)
        return np.log1p(df[CFG.TARGET_COLS].values.astype(np.float32))

In [26]:
# ==========================================
# 4. DATASET & TRANSFORMS
# ==========================================
train_transforms = transforms.Compose([
    transforms.Resize((CFG.IMG_SIZE, CFG.IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_transforms = transforms.Compose([
    transforms.Resize((CFG.IMG_SIZE, CFG.IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

class BiomassDataset(Dataset):
    def __init__(self, X_tab, image_paths, y, transform=None):
        self.X_tab = torch.tensor(X_tab, dtype=torch.float32)
        self.image_paths = image_paths
        self.y = torch.tensor(y, dtype=torch.float32)
        self.transform = transform

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        # Tabular
        x_tab = self.X_tab[idx]
        
        # Image
        path = self.image_paths[idx]
        try:
            image = Image.open(path).convert("RGB")
        except Exception as e:
            # Fallback for missing images
            image = Image.new('RGB', (CFG.IMG_SIZE, CFG.IMG_SIZE), (0, 0, 0))

        if self.transform:
            image = self.transform(image)
            
        return (x_tab, image), self.y[idx]

In [27]:
# ==========================================
# 5. MODEL ARCHITECTURE
# ==========================================
class BiomassModel(pl.LightningModule):
    def __init__(self, tabular_input_dim, target_dim=3):
        super().__init__()
        self.save_hyperparameters()

        # Setup weights for the Loss
        weights = torch.tensor([CFG.TARGET_WEIGHTS[t] for t in CFG.TARGET_COLS], dtype=torch.float32)
        # Weights normalized to sum to 1
        self.target_loss_weights = weights / weights.sum()

        # 1. Vision Backbone (DINOv2)
        self.dino = timm.create_model(CFG.MODEL_NAME, pretrained=True, num_classes=0, img_size=CFG.IMG_SIZE)
        
        # FREEZE Backbone initially to prevent overfitting/catastrophic forgetting
        for param in self.dino.parameters():
            param.requires_grad = False
            
        dino_out_dim = self.dino.num_features

        # 2. Tabular Branch
        self.tabular_branch = nn.Sequential(
            nn.Linear(tabular_input_dim, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
        )

        # 3. Fusion Head
        self.head = nn.Sequential(
            nn.Linear(dino_out_dim + 64, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, target_dim)
        )
        
        # Huber Loss is robust to outliers, reduction='none' for custom weighting
        self.base_criterion = nn.HuberLoss(delta=1.0, reduction='none')

    # Function to calculate weighted loss
    def calculate_weighted_loss(self, y_pred, y_true):
        # Calculate base loss per element
        loss_per_element = self.base_criterion(y_pred, y_true)
        
        # Apply target weights (Broadcasting)
        # The weights tensor has shape [1, 5] and is multiplied by loss [BatchSize, 5]
        weighted_loss = loss_per_element * self.target_loss_weights.to(self.device)
        
        # Return the mean of the weighted loss
        return weighted_loss.mean()

    def forward(self, x_tab, x_img):
        # DINO inference (no grad required for backbone)
        with torch.no_grad():
            img_feat = self.dino(x_img)
        
        tab_feat = self.tabular_branch(x_tab)
        combined = torch.cat([img_feat, tab_feat], dim=1)
        return self.head(combined)

    def training_step(self, batch, batch_idx):
        (x_tab, x_img), y = batch
        y_pred = self(x_tab, x_img)
        loss = self.calculate_weighted_loss(y_pred, y)
        self.log('train_loss', loss, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        (x_tab, x_img), y = batch
        y_pred = self(x_tab, x_img)
        loss = self.calculate_weighted_loss(y_pred, y)
        self.log('val_loss', loss, prog_bar=True)
        return loss

    def configure_optimizers(self):
        # Filltering backbone parameters that belong to the DINOv2 backbone
        backbone_params = [p for n, p in self.named_parameters() if 'dino' in n and p.requires_grad]
        
        # Head and Tabular Branch parameters
        head_and_tabular_params = [p for n, p in self.named_parameters() if ('head' in n or 'tabular_branch' in n) and p.requires_grad]
        
        # Define two parameter groups with different LRs
        # higher for head and tabular branch, lower for backbone
        param_groups = [
            {'params': head_and_tabular_params, 'lr': CFG.LR_HEAD, 'weight_decay': CFG.WEIGHT_DECAY},
            {'params': backbone_params, 'lr': CFG.LR_BACKBONE, 'weight_decay': CFG.WEIGHT_DECAY},
        ]
        
        # Define optimizer with parameter groups
        optimizer = torch.optim.AdamW(param_groups)
        
        # Define scheduler
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer, mode='min', factor=0.5, patience=3
        )
        return {
            "optimizer": optimizer,
            "lr_scheduler": {"scheduler": scheduler, "monitor": "val_loss"}
        }


In [28]:
# ==========================================
# 6. MAIN TRAINING LOOP (K-FOLD)
# ==========================================
if __name__ == "__main__":
    # Load Data
    train_df_full = load_and_preprocess_data(CFG.TRAIN_PATH)
    
    # Setup GroupKFold using Sampling_Date
    gkf = GroupKFold(n_splits=CFG.N_FOLDS)
    groups = train_df_full['Sampling_Date'] 

    
    print(f"\n🚀 Starting training with {CFG.MODEL_NAME}...")

    # Best model 
    overall_best_loss = float('inf')
    overall_best_fold_idx = -1
    overall_best_checkpoint_path = None
    all_fold_results = []
    
    for fold, (train_idx, val_idx) in enumerate(gkf.split(train_df_full, groups=groups)):
        print(f"\n{'='*20} FOLD {fold+1}/{CFG.N_FOLDS} {'='*20}")
        
        # 1. Split Data
        train_df = train_df_full.iloc[train_idx].reset_index(drop=True)
        val_df = train_df_full.iloc[val_idx].reset_index(drop=True)
        
        # 2. Preprocessing (Fit on Train, Transform Val)
        processor = BiomassPreprocessor()
        
        # Features
        X_train_tab = processor.fit_transform(train_df)
        X_val_tab = processor.transform(val_df)
        
        # Targets (Log Transformed)
        y_train = processor.transform_targets(train_df)
        y_val = processor.transform_targets(val_df)
        
        # 3. Create Datasets & Loaders
        train_ds = BiomassDataset(X_train_tab, train_df['full_image_path'].values, y_train, transform=train_transforms)
        val_ds = BiomassDataset(X_val_tab, val_df['full_image_path'].values, y_val, transform=val_transforms)
        
        train_loader = DataLoader(train_ds, batch_size=CFG.BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True, drop_last=True)
        val_loader = DataLoader(val_ds, batch_size=CFG.BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True, drop_last=True)
        
        # 4. Initialize Model
        model = BiomassModel(tabular_input_dim=X_train_tab.shape[1], target_dim=len(CFG.TARGET_COLS))
        
        # 5. Setup Trainer
        checkpoint_callback = ModelCheckpoint(
            dirpath=f'checkpoints/fold_{fold}',
            filename='best-{epoch:02d}-{val_loss:.4f}',
            monitor='val_loss',
            mode='min',
            save_top_k=1
        )
        
        early_stop = EarlyStopping(monitor='val_loss', patience=5, mode='min')
        
        trainer = pl.Trainer(
            max_epochs=CFG.EPOCHS,
            accelerator="auto",
            devices=1,
            precision="16-mixed",  # Uses FP16 to save memory & speed up
            callbacks=[checkpoint_callback, early_stop],
            logger=False,          # Set to True if using WandB/Tensorboard
            enable_progress_bar=True
        )
        
        # 6. Train
        trainer.fit(model, train_loader, val_loader)


        print("\n Unfreezing Backbone for Fine-Tuning...")

        # Unfreeze last N blocks of DINOv2 Backbone
        # Start from the end of the blocks list
        for i in range(1, CFG.NUM_BLOCKS_TO_UNFREEZE + 1):
             for param in model.dino.blocks[-i].parameters():
                 param.requires_grad = True

        # Unfreeze Batch Norm layers which are often problematic if frozen
        for param in model.dino.norm.parameters():
            param.requires_grad = True
        
        print(f"Unfroze the last {CFG.NUM_BLOCKS_TO_UNFREEZE} blocks of the Backbone.")
        
        # New set of callbacks for Fine-Tuning with shorter EarlyStopping
        checkpoint_callback_ft = ModelCheckpoint(
            dirpath=f'checkpoints/fold_{fold}/finetune',
            filename='best-ft-{epoch:02d}-{val_loss:.4f}',
            monitor='val_loss',
            mode='min',
            save_top_k=1
        )
        early_stop_ft = EarlyStopping(monitor='val_loss', patience=3, mode='min') 
        
        # Fine Tuning Phase with a newer Trainer with reduced epochs
        trainer_ft = pl.Trainer(
            max_epochs=CFG.EPOCHS_FT,
            devices=1,
            precision="16-mixed",
            callbacks=[checkpoint_callback_ft, early_stop_ft],
            logger=False, 
            enable_progress_bar=True
        )
        
        # Train Fine-Tuning
        trainer_ft.fit(model, train_loader, val_loader)
        
        # Select the best score between Warm-up and Fine-Tuning
        best_warmup_loss = checkpoint_callback.best_model_score.item() if checkpoint_callback.best_model_score is not None else float('inf')
        best_finetune_loss = checkpoint_callback_ft.best_model_score.item() if checkpoint_callback_ft.best_model_score is not None else float('inf')

        if best_warmup_loss <= best_finetune_loss:
            best_loss_current_fold = best_warmup_loss
            best_path_current_fold = checkpoint_callback.best_model_path
        else:
            best_loss_current_fold = best_finetune_loss
            best_path_current_fold = checkpoint_callback_ft.best_model_path


        print(f"Fold {fold+1} Complete.")
        print(f"Best Warm-up Loss: {best_warmup_loss:.4f}")
        print(f"Best Fine-Tuning Loss: {best_finetune_loss:.4f}")
        print(f"Best Global Loss for Fold {fold+1}: {best_loss_current_fold:.4f}")

        # Update the best global model
        if best_loss_current_fold < overall_best_loss:
            overall_best_loss = best_loss_current_fold
            overall_best_fold_idx = fold
            overall_best_checkpoint_path = best_path_current_fold

        # Memorize results for R2
        all_fold_results.append({
            'fold': fold,
            'best_loss': best_loss_current_fold,
            'path': best_path_current_fold,
            'train_idx': train_idx,
            'val_idx': val_idx
        })


    print("\n🎉 GroupKFold Cross-Validation Completed Successfully!")
    print(f"\n⭐ Overall Best Model is from FOLD {overall_best_fold_idx+1} with Loss: {overall_best_loss:.4f}")
    print(f"Best Checkpoint Path: {overall_best_checkpoint_path}")

   

📂 Loading and formatting data...
✅ Loaded 357 unique train samples.

🚀 Starting training with vit_base_patch14_dinov2.lvd142m...

==================== FOLD 1/5 ====================


Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/usr/local/lib/python3.11/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:751: Checkpoint directory /kaggle/working/checkpoints/fold_0 exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.11/dist-packages/pytorch_lightning/utilities/model_summary/model_summary.py:231: Precision 16-mixed is not supported by the model summary.  Estimated model size in MB will not be accurate. Using 32 bits instead.

  | Name           | Type              | Params | Mode 
-------------------------------------------------------------
0 | dino           | VisionTransformer | 86.6 M | train
1 | tabular_branch | Sequential        | 11.6 K | train
2 | head           | Sequential        | 215 K  | train
3 | base_criterion | HuberLoss         | 0      | train
----------------------------------------------

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=5` reached.


Fold 5 Complete.
Best Warm-up Loss: 0.0634
Best Fine-Tuning Loss: 0.1160
Best Global Loss for Fold 5: 0.0634

🎉 GroupKFold Cross-Validation Completed Successfully!

⭐ Overall Best Model is from FOLD 2 with Loss: 0.0436
Best Checkpoint Path: /kaggle/working/checkpoints/fold_1/best-epoch=18-val_loss=0.0436.ckpt

📏 Calculating R-squared (R2) score on the validation set of the best fold...


NameError: name 'r2_score' is not defined

In [32]:
 # Compute R2 score
if overall_best_checkpoint_path and overall_best_fold_idx != -1:
    # Select data and indices of the best fold
    best_result = [r for r in all_fold_results if r['fold'] == overall_best_fold_idx][0]
    train_idx = best_result['train_idx']
    val_idx = best_result['val_idx']

    print("\n📏 Calculating R-squared (R2) score on the validation set of the best fold...")

    # 1. Restore the datasets
    train_df = train_df_full.iloc[train_idx].reset_index(drop=True)
    val_df = train_df_full.iloc[val_idx].reset_index(drop=True)

    # 2. Preprocessing (Fit on Train, Transform on Val)
    processor = BiomassPreprocessor()
    X_train_tab = processor.fit_transform(train_df) # Fit on train
    X_val_tab = processor.transform(val_df)         # Transform on val
    y_val = processor.transform_targets(val_df)     # Real validation targets (Log Transformed)
    # 3. Create the Validation DataLoader (new, drop_last=False, as done above)
    val_ds = BiomassDataset(X_val_tab, val_df['full_image_path'].values, y_val, transform=val_transforms)
    val_loader = DataLoader(val_ds, batch_size=CFG.BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True, drop_last=False)

    # 4. Load the Model and Predict
    tabular_input_dim = X_train_tab.shape[1]
    target_dim = len(CFG.TARGET_COLS)

    # Load the model from the checkpoint
    model = BiomassModel.load_from_checkpoint(
        overall_best_checkpoint_path,
        tabular_input_dim=tabular_input_dim,
        target_dim=target_dim,
        strict=False
    )

    # Prepare for inference
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()

    all_preds = []
    all_targets = []

    with torch.no_grad():
        for (x_tab, x_img), y in val_loader:
            x_tab = x_tab.to(device)
            x_img = x_img.to(device)
            y_pred = model(x_tab, x_img)

            all_preds.append(y_pred.cpu().numpy())
            all_targets.append(y.cpu().numpy())

    y_pred_np = np.concatenate(all_preds)
    y_true_np = np.concatenate(all_targets)

    # 5. Calculate the R-squared (R2) Score
    # multioutput='variance_weighted' is appropriate for multi-target metrics
    r2 = r2_score(y_true_np, y_pred_np, multioutput='variance_weighted')

    print(f"\n🎉 Final Result: R-squared (R2) score for the Best Model: {r2:.4f}")
else:
    print("\n⚠️ Unable to perform R2 evaluation: No model checkpoint was saved or found.")


📏 Calculating R-squared (R2) score on the validation set of the best fold...

🎉 Final Result: R-squared (R2) score for the Best Model: 0.3096
